In [2]:
from decodes.core import *
from decodes.io.jupyter_out import JupyterOut
import math

out = JupyterOut.unit_square( )

http://decod.es/	v0.2.3
io loaded


# Graph Objects in Decod.es
If our vector-based point of view is stretched to account for meshes and rasters, the subject of this section, the graph, breaks the utility of a vector approach entirely.

***A graph is understood as a representation of a network of nodes and connections.*** The value of graphs lies in their capacity to reveal essential structures within a set of relationships.

<img src="http://geometric-computation-images.s3-website-us-east-1.amazonaws.com/1.07.C08.jpg" style="width: 200px; display: inline;">


## Elements of a Graph

A graph is made up of a set of discrete elements called ***nodes*** that are associated with objects, and a set of ***edges*** that associate pairs of nodes. 

These edges may be ***directional***, in which case they are often depicted with the addition of an arrow, or ***non-directional***. The Decod.es Graph is implemented as a directional manner, but allows relationships to be defined in a way that adds two edges at a time, thereby acting as bi-directional or non-directional.



In Decod.es, a Graph is defined as an object with three members, each of which is a collection type that we have not often seen applied:

* `gph.nodes` describes the nodes of a Graph as a Python Set of unique arbitrary objects
* `gph.edges` describes the relationships between nodes using a Python Dict
* `gph.weights` also uses a Python Dict, and describes the relative weight of each edge of the Graph.

<img src="http://geometric-computation-images.s3-website-us-east-1.amazonaws.com/1.07.C02.jpg" style="width: 600px; display: inline;">


The `gph.nodes` member relies on the nature of the Python Set to ensure that no duplicate nodes may exist on the Graph. 

This enforced uniqueness allows us to structure `gph.edges` as a Dict that uses objects known to be stored in `gph.nodes` as its keys, and to thereby associate each node with a dictionary entry that contains all the other nodes to which it is connected. 

Structuring the `gph.edges` Dict in such a manner makes retrieving all the edges related to a given node nod a simple matter of calling `gph.edges[nod]`. 

Weights are similarly structured as a dictionary, but are ***keyed by pairs of nodes*** that are associated with numeric weights. 

As an example of using these structures together, to find the weight of the first edge that originates at node `nod`, we would pass a Tuple of the given node and the partner indicated by the first item of the edges List, as in `gph.weights[(nod, gph.edges[nod][0])]`

<img src="http://geometric-computation-images.s3-website-us-east-1.amazonaws.com/1.07.P34.jpg" style="width: 200px; display: inline;">


An example. Imagine that we are exploring the geographical distribution of major American cities, and want to better understand how clusters of large cities are connected to one another. We could explore these relationships in a graph, which might illuminate these relationships more clearly by describing each city as a node, and the driving time between two cities as a weighted edge.

<img src="http://geometric-computation-images.s3-website-us-east-1.amazonaws.com/1.07.P09.jpg" style="width: 200px; display: inline;">


If each city is described by an abbreviated String, then this data would manifest as a Decod.es Graph that stores Strings as nodes, and relates Strings to other Strings by edges that are weighted by Integers that describe driving distances.

In this case, `gph.edges` is a Dict keyed by Strings found in `gph.nodes`, and which returns other such Strings. Similarly, `gph.weights` is a Dict keyed by pairs of Strings found in `gph.nodes`, and which returns numeric values.

To build such a construction, a Graph must be initialized and populated. This is conveniently done using the `gph.add_edge()` method of an existing Graph, which automatically handles the addition of nodes, edges, and weights as needed. Two nodes must be provided to this method (a from-node and a to-node) as well as an optional weight for the resulting edge.

In [3]:
"""
Initializing and Populating a Graph
The add_edge method orchestrates the addition of objects to the three members 
of a Graph. First, if the given nodes are not yet stored in the gph.nodes Set, 
they are added. Next, two relationships are defined in the gph.edges Dict if 
the edge is bidirectional and just one is added if the edge is not. Finally, 
the weight of the edge is logged in the gph.weights member.
"""
gph = Graph()
gph.add_edge("LAX","NYC",42)
gph.add_edge("CHI","NYC",13)
gph.add_edge("CHI","LAX",30)
gph.add_edge("HOU","NYC",24)


True

Given a populated Graph, we may access the edges associated with any node by passing the appropriate key to the gph.edges Dict.

Notice in the output that results from the code below, that the “NYC” node is the origin for edges connected to three other nodes, even though the `gph.add_edge()` method was never called with “NYC” as a from-node. This behavior reflects that of a bidirectional graph, wherein each edge is assumed to describe a two-way relationship.

In [5]:
"""
Accessing Graph Edges
A List of edges associated with a given object may be accessed by passing the 
object as a key to the gph.edges Dict.
"""
print (gph.edges["NYC"])

['LAX', 'CHI', 'HOU']


The use of a Python Dict to describe Graph edges allows us to take advantage of the `items()` method to iterate over pairs of nodes. The code seen below is identical to that of the gph.node_pairs property.

In [4]:
"""
Iterating Graph Edges
To iterate over all the edges of a Graph, we may rely on the edges.items() 
method, which returns key-value pairs. Note that in the case of bidirectional 
edges, this method produces duplicate pairs of nodes in reverse order. This 
procedure is implemented as the gph.node_pairs property.
"""
for nod_a, others in gph.edges.items():
    for nod_b in others:
        print nod_a, " -> ", nod_b

NYC  ->  LAX
NYC  ->  CHI
NYC  ->  HOU
HOU  ->  NYC
LAX  ->  NYC
LAX  ->  CHI
CHI  ->  NYC
CHI  ->  LAX


<img src="http://geometric-computation-images.s3-website-us-east-1.amazonaws.com/1.07.P08.jpg" style="width: 600px; display: inline;">


The following routine demonstrates how to access the weight of a given edge.

Here, we iterate over each pair of nodes that are related by an edge on our Graph. Since the `gph.weights` member is a Dict that is keyed by exactly this pairing, access to the weight of each edge is quite simple. Note that the order in which the below code iterates over pairs of nodes is not the same as the order in which nodes were added to our Graph via the `gph.add_edge()` method. This is due to the unordered nature of both collection types involved; both Dicts and Sets do not offer a means to retrieve stored data in order.

In [5]:
"""
Accessing Graph Weights
By using the gph.node_pairs property to iterate the edges of a Graph, we can 
generate pairs of nodes that may be used to access edge weights.
"""
for nod_a, nod_b in gph.node_pairs:
    str = "driving from {} to {} takes {} hours"
    print str.format(nod_a, nod_b, gph.weights[(nod_a,nod_b)])

driving from NYC to LAX takes 42 hours
driving from NYC to CHI takes 13 hours
driving from NYC to HOU takes 24 hours
driving from HOU to NYC takes 24 hours
driving from LAX to NYC takes 42 hours
driving from LAX to CHI takes 30 hours
driving from CHI to NYC takes 13 hours
driving from CHI to LAX takes 30 hours
